# Importing Libraries 

In [4]:
import pandas as pd 
import numpy as np 
import os 

# Import Dataframe

In [6]:
# Path Variable 
path = r'C:\Users\jmiel\JENNY_STUFF\MY_DOCS\CareerFoundry\Instacart Basket Analysis'

# Import ords_prods_merge dataframe 
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_new_columns.pkl'))

# Task 4.8 

In [8]:
ords_prods_merge.shape

(32404859, 18)

In [9]:
# Find aggregated mean of the order_number column grouped by department_id 
ords_prods_merge.groupby('department_id')['order_number'].mean() 

department_id
1     15.457838
2     17.277920
3     17.170395
4     17.811403
5     15.215751
6     16.439806
7     17.225802
8     15.340650
9     15.895474
10    20.197148
11    16.170638
12    15.887671
13    16.583536
14    16.773669
15    16.165037
16    17.665606
17    15.694469
18    19.310397
19    17.177343
20    16.473447
21    22.902379
Name: order_number, dtype: float64

After comparing the subset results to the entire dataframe, the values did not seem to change significantly. 
Some numerical values remained pretty similar, while others were slightly higher / lower compared to the subset. 

In [11]:
# Aggregating data with transform() / create a loyalty flag
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

C:\Users\jmiel\AppData\Local\Temp\ipykernel_7940\1238283728.py:2: FutureWarning: The provided callable <function max at 0x00000139A70EECA0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)


In [12]:
ords_prods_merge.head(15)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_days,busiest_period_of_day,max_order
0,2539329,1,1,2,8,NaN,True,196,1,0,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Average orders,10
1,2539329,1,1,2,8,NaN,True,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16,12.5,both,Mid-range product,Regularly busy,Average orders,10
2,2539329,1,1,2,8,NaN,True,12427,3,0,Original Beef Jerky,23,19,4.4,both,Low-range product,Regularly busy,Average orders,10
3,2539329,1,1,2,8,NaN,True,26088,4,0,Aged White Cheddar Popcorn,23,19,4.7,both,Low-range product,Regularly busy,Average orders,10
4,2539329,1,1,2,8,NaN,True,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54,17,1.0,both,Low-range product,Regularly busy,Average orders,10
5,2398795,1,2,3,7,15.0,False,196,1,1,Soda,77,7,9.0,both,Mid-range product,Slowest days,Average orders,10
6,2398795,1,2,3,7,15.0,False,10258,2,0,Pistachios,117,19,3.0,both,Low-range product,Slowest days,Average orders,10
7,2398795,1,2,3,7,15.0,False,12427,3,1,Original Beef Jerky,23,19,4.4,both,Low-range product,Slowest days,Average orders,10
8,2398795,1,2,3,7,15.0,False,13176,4,0,Bag of Organic Bananas,24,4,10.3,both,Mid-range product,Slowest days,Average orders,10
9,2398795,1,2,3,7,15.0,False,26088,5,1,Aged White Cheddar Popcorn,23,19,4.7,both,Low-range product,Slowest days,Average orders,10


In [13]:
# Deriving columns with loc() 
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [14]:
# Check frequency of loyalty_flag column 
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: count, dtype: int64

In [15]:
ords_prods_merge['loyalty_flag'].head()

0    New customer
1    New customer
2    New customer
3    New customer
4    New customer
Name: loyalty_flag, dtype: object

In [16]:
# Spending habits of each loyalty category (min, max, average)
ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

It appears as though a new customer has an average that is $3 more than that of the loyal customer. 
A regular customer is a dollar less than a new customer. The least average spent is from the loyal customer. 

In [18]:
# Create a new column to prepare for creating flag 
ords_prods_merge['average_spent'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

C:\Users\jmiel\AppData\Local\Temp\ipykernel_7940\150403192.py:2: FutureWarning: The provided callable <function mean at 0x00000139A70EF6A0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ords_prods_merge['average_spent'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)


In [19]:
pd.options.display.max_rows = None

In [ ]:
# Check output 
ords_prods_merge[['user_id', 'average_spent','loyalty_flag']].head(100)

In [21]:
# Create a spent_flag for high and low spenders 
ords_prods_merge.loc[ords_prods_merge['average_spent'] < 10, 'spent_flag'] = 'Low spender'
ords_prods_merge.loc[ords_prods_merge['average_spent'] >= 10, 'spent_flag'] = 'High spender'

In [22]:
# Frequency check 
ords_prods_merge['spent_flag'].value_counts(dropna = False)

spent_flag
Low spender     31770614
High spender      634245
Name: count, dtype: int64

In [23]:
ords_prods_merge[['user_id', 'average_spent', 'spent_flag']].head(20)

,user_id,average_spent,spent_flag
0,1,6.367797,Low spender
1,1,6.367797,Low spender
2,1,6.367797,Low spender
3,1,6.367797,Low spender
4,1,6.367797,Low spender
5,1,6.367797,Low spender
6,1,6.367797,Low spender
7,1,6.367797,Low spender
8,1,6.367797,Low spender
9,1,6.367797,Low spender


In [24]:
# Create a column on customer_frequency 
ords_prods_merge['customer_frequency'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

C:\Users\jmiel\AppData\Local\Temp\ipykernel_7940\497339069.py:2: FutureWarning: The provided callable <function median at 0x00000139A7246E80> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  ords_prods_merge['customer_frequency'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)


In [25]:
# Create a flag on customer frequency 
ords_prods_merge.loc[ords_prods_merge['customer_frequency'] > 20, 'frequency_flag'] = 'Non-frequent customer'
ords_prods_merge.loc[(ords_prods_merge['customer_frequency'] > 10) & (ords_prods_merge['customer_frequency'] <= 20), 'frequency_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['customer_frequency'] <= 10, 'frequency_flag'] = 'Frequent customer'

In [26]:
# Frequency check 
ords_prods_merge['frequency_flag'].value_counts(dropna = False)

frequency_flag
Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
NaN                             5
Name: count, dtype: int64

The NaN column should indicate that these were first time customers 

In [28]:
# View necessary columns
ords_prods_merge[['user_id', 'days_since_prior_order', 'average_spent', 'frequency_flag']].head(20)

,user_id,days_since_prior_order,average_spent,frequency_flag
0,1,NaN,6.367797,Non-frequent customer
1,1,NaN,6.367797,Non-frequent customer
2,1,NaN,6.367797,Non-frequent customer
3,1,NaN,6.367797,Non-frequent customer
4,1,NaN,6.367797,Non-frequent customer
5,1,15.0,6.367797,Non-frequent customer
6,1,15.0,6.367797,Non-frequent customer
7,1,15.0,6.367797,Non-frequent customer
8,1,15.0,6.367797,Non-frequent customer
9,1,15.0,6.367797,Non-frequent customer


# Export dataframe 


In [51]:
ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_merge_aggregated.pkl'))